# Домашнее задание «Функции и работа с данными»

## Комментарий преподавателя
Добрый вечер, спасибо за решения.

В первом задании в функции class_movies проверка if row['rating'] <= 2 дублируется elif 2 < row['rating'] <= 4 в следующей строке. Ошибки тут нет, но если придется что-то менять, то надо будет не забыть внести изменения во всех строчках. Повышается вероятность случайно ошибиться.

Во втором задании лучше проходить по регионам словаря geo_data в цикле. А то регионов может быть много, все будет сложно перечислять вручную в коде:
```
def region(keyword):

# проходим по ключам (название региона) и значениям (список городов региона) словаря geo_data
for region, city_list in geo_data.items():
    # проходим по каждому городу региона
    for city in city_list:
        # если название города есть в поисковом запросе, то возвращаем название региона region
        if city in keyword:
            return region

# если ничего не нашлось, то возвращаем undefined
return 'undefined'
```

В последнем задании в функции production_year можно вынести расчет re.findall(r'.*(\d{4}).*', row) в отдельную переменную, чтобы не повторять одинаковый расчет несколько раз.

**Домашнее задание к лекции.

## Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd



In [2]:
# Task 1

ratings =  pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')


In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
def class_movies(row):
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif 2 < row['rating'] <= 4:
        return 'средний рейтинг'
    elif 4.5 <= row['rating'] <= 5:
        return 'высокий рейтинг'
    
ratings['class'] = ratings.apply(class_movies, axis=1)
movies_rating = ratings.merge(movies, on='movieId', how='left')
movies_rating.head()


,userId,movieId,rating,timestamp,class,title,genres
0,1,31,2.5,1260759144,средний рейтинг,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,средний рейтинг,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,средний рейтинг,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,низкий рейтинг,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,средний рейтинг,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

1
2
3
4
5
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [36]:
# Task 2

keywords = pd.read_csv('keywords.csv')

geo_data = {'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

def region(row):
    '''Классифицирует города по регионам'''
    if row['keyword'] in geo_data['Центр']:
        return 'Центр'
    elif row['keyword'] in geo_data['Северо-Запад']:
        return 'Северо-Запад'
    elif row['keyword'] in geo_data['Дальний Восток']:
        return 'Дальний Восток'
    else:
        return 'undefined'

# результат записывается в столбец region
keywords['region'] = keywords.apply(region, axis=1)

keywords[keywords['region'] != 'undefined'].head()

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

  1. для каждой строки пройдите по всем годам списка years
  2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

P/S/ все файлы находятся в архиве «дополнительные файлы к лекции и дз»

Важно! При копировании юпитер ноутбука изменяет кавычки ’ на ` , что может способствовать ошибки в коде. В данной ситуации необходимо их просто заменить.

In [7]:
movies_rating.head()

,userId,movieId,rating,timestamp,class,title,genres
0,1,31,2.5,1260759144,средний рейтинг,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,средний рейтинг,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,средний рейтинг,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,низкий рейтинг,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,средний рейтинг,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [33]:
import re
def production_year(row):
    '''Выделяет из строки названия фильма год выпуска'''
    #year = int(re.findall(r'.*(\d{4}).*', row))
    
    if re.findall(r'.*(\d{4}).*', row):
        if 1950 <= int(re.findall(r'.*(\d{4}).*', row)[0]) <= 2010:
            return int(re.findall(r'.*(\d{4}).*', row)[0])
        else:
            return 1900
    else:
        return 1900

# год выпуска записываеся в столбец years
movies_rating['years'] = movies_rating['title'].apply(production_year)

movies_rating.head()


,userId,movieId,rating,timestamp,class,title,genres,years
0,1,31,2.5,1260759144,средний рейтинг,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,средний рейтинг,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,средний рейтинг,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,низкий рейтинг,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,средний рейтинг,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [38]:
# группировка по годам и подсчет среднего значения с сортировкой рейтинга по убыванию
movies_rating.groupby(['years']).mean().sort_values('rating', ascending=False).head(10)

,userId,movieId,rating,timestamp
years,,,,
1957,360.933544,2750.965190,4.014241,1.083707e+09
1972,359.694878,3983.538976,4.011136,1.122759e+09
1952,346.394737,4107.796053,4.000000,1.090512e+09
1954,358.228324,2867.661850,3.994220,1.070591e+09
1951,347.106996,2605.588477,3.983539,1.052714e+09
1974,357.181481,3956.801852,3.978704,1.112526e+09
1962,350.671196,3479.100543,3.952446,1.094200e+09
1950,355.720339,3503.427966,3.915254,1.090648e+09
1977,351.040059,3604.516320,3.905786,1.116007e+09
